### Establishing BigQuery Connection

Includes account key via the json file stored in the folder which contains the project repo

In [3]:
from google.cloud import bigquery

import pathlib # __file__ isn't recognized in Jupyter, so we need this to get the root path
path = pathlib.Path.cwd()
PROJ_ROOT = path.parent

accountkeyfile = str(PROJ_ROOT.parent / 'credentials\GoogleServiceAccountKeyFile.json')
client = bigquery.Client.from_service_account_json(accountkeyfile)

#### Accounts Per Device

Device determined by device.advertising_id

In [2]:
query = """
SELECT
    device.advertising_id AS device_id,
    COUNT(DISTINCT user_properties.value.string_value) AS Usernames,
    COUNT(DISTINCT user_pseudo_id) AS Pseudo_IDs
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE 
    ((SELECT event_params.value.string_value
        FROM UNNEST(event_params) AS event_params
        WHERE event_params.key = 'firebase_screen_class') = 'HomeActivity')
GROUP BY
    device.advertising_id
ORDER BY 
    COUNT(DISTINCT user_properties.value.string_value) DESC
"""
df = client.query(query).to_dataframe()
df.head()

,device_id,Usernames,Pseudo_IDs
0,733bdd53-745a-4e95-b32e-9595690aaf52,20,29
1,788dc562-cc05-44c0-bc26-7e24ff3c3768,16,36
2,42479d6a-a2e2-4530-be2f-1cc0a77245f0,3,4
3,b3c6b09d-e8b0-413d-a285-bce02fdfb58c,2,2
4,2f84376a-841a-4d6f-83fc-2d339294e02d,2,2


### Total # of Events by Phone model

In [12]:
query = """
SELECT
    COUNT(device.mobile_os_hardware_model) AS events,
    device.mobile_os_hardware_model AS model
FROM
    `heycharlie-ada47.analytics_153084895.events_*`
WHERE
    event_date > '20181109'
GROUP BY
    device.mobile_os_hardware_model
ORDER BY
    COUNT(device.mobile_os_hardware_model) DESC
"""
df = client.query(query).to_dataframe()
df.head()

,events,model
0,103534,SM-N920V
1,40713,Pixel XL
2,40450,SAMSUNG-SM-G891A
3,31763,SM-G930V
4,31462,SM-G950U


### Total # of Events by Username

In [14]:
query = """
SELECT
    COUNT(user_properties.value.string_value) AS events,
    user_properties.value.string_value AS Username
FROM
    `heycharlie-ada47.analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE
    event_date > '20181109'
GROUP BY
    user_properties.value.string_value
ORDER BY
    COUNT(user_properties.value.string_value) DESC
"""
df = client.query(query).to_dataframe()
df.head()

,events,Username
0,51762,Rabbit
1,19876,emily
2,15674,Jake
3,11054,anonysauce
4,8495,mckay68


### # of Notification Events by Username

TODO - Break out notifications by type (positive reinforcement 1x/day, texts, calls)

TODO - Look at responses to notifications

In [17]:
query = """
SELECT
  user_properties.value.string_value as username,
  COUNT(*) AS count
FROM
  `analytics_153084895.events_*`,
  UNNEST(user_properties) AS user_properties,
  UNNEST(event_params) AS event_params
WHERE 
  event_params.key = 'notification_event'
  AND event_date > '20181122'
  AND user_properties.value.string_value != 'null'
GROUP BY
  user_properties.value.string_value
ORDER BY
  COUNT(*) DESC
"""
df = client.query(query).to_dataframe()
df.head()

,username,count
0,mckay68,1249
1,emily,309
2,+vinny,220
3,anonysauce,175
4,username,156


### Daily Count of Risky Text Events

In [19]:
query = """
SELECT
    event_date AS Date,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_event" THEN 1 ELSE 0 END) AS risky_sent_sms,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_sent_notification,
    SUM(CASE WHEN event_name = "risky_sms_event" THEN 1 ELSE 0 END) AS risky_in_sms,
    SUM(CASE WHEN event_name = "risky_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_in_notification,
    COUNT(event_name) AS events
FROM 
    (SELECT DISTINCT event_date, event_name, event_server_timestamp_offset
        FROM `analytics_153084895.events_*`)
WHERE
    event_name = "risky_sent_sms_warning_event"
    OR event_name = "risky_sent_sms_warning_notification"
    OR event_name = "risky_sms_event"
    OR event_name = "risky_sms_warning_notification"
GROUP BY
    event_date
ORDER BY
    event_date DESC
"""
df = client.query(query).to_dataframe()
df.head()

,Date,risky_sent_sms,risky_sent_notification,risky_in_sms,risky_in_notification,event
0,20190123,1,1,1,1,4
1,20190119,6,4,6,4,20
2,20190118,0,0,1,1,2
3,20190117,4,3,7,7,21
4,20190116,3,3,5,3,14


### Daily Risky Events and Notifications, by User

In [22]:
query = """
SELECT
    event_date AS Date,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_event" THEN 1 ELSE 0 END) AS risky_sent_sms,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_sent_notification,
    SUM(CASE WHEN event_name = "risky_sms_event" THEN 1 ELSE 0 END) AS risky_in_sms,
    SUM(CASE WHEN event_name = "risky_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_in_notification,
    user_properties.value.string_value AS username,
    COUNT(event_name) AS total_risly_sms_events
FROM 
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE
    user_properties.value.string_value <> "null"
    AND (event_name = "risky_sent_sms_warning_event"
        OR event_name = "risky_sent_sms_warning_notification"
        OR event_name = "risky_sms_event"
        OR event_name = "risky_sms_warning_notification")
GROUP BY
    event_date, user_properties.value.string_value
ORDER BY
    event_date DESC
"""
df = client.query(query).to_dataframe()
df.head(10)

,Date,risky_sent_sms,risky_sent_notification,risky_in_sms,risky_in_notification,username,events
0,20190123,3,1,15,3,emily,22
1,20190119,18,4,28,4,vinnov10,54
2,20190118,0,0,1,1,Jake,2
3,20190117,0,0,5,4,vinnov10,9
4,20190117,3,2,22,5,emily,32
5,20190117,1,1,2,0,hbickers,4
6,20190116,8,4,60,4,emily,76
7,20190116,1,1,6,0,liamkl,8
8,20190115,1,0,6,2,hbickers,9
9,20190115,4,3,14,2,emily,23


### Days of App Use

In [ ]:
query = """
SELECT
    event_date AS Date,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_event" THEN 1 ELSE 0 END) AS risky_sent_sms,
    SUM(CASE WHEN event_name = "risky_sent_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_sent_notification,
    SUM(CASE WHEN event_name = "risky_sms_event" THEN 1 ELSE 0 END) AS risky_in_sms,
    SUM(CASE WHEN event_name = "risky_sms_warning_notification" THEN 1 ELSE 0 END) AS risky_in_notification,
    user_properties.value.string_value AS username,
    COUNT(event_name) AS total_risly_sms_events
FROM 
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE
    user_properties.value.string_value <> "null"
    AND (event_name = "risky_sent_sms_warning_event"
        OR event_name = "risky_sent_sms_warning_notification"
        OR event_name = "risky_sms_event"
        OR event_name = "risky_sms_warning_notification")
GROUP BY
    event_date, user_properties.value.string_value
ORDER BY
    event_date DESC
"""
df = client.query(query).to_dataframe()
df.head(10)

### Days of App Use

In [23]:
query = """
SELECT
    MIN(event_date) AS Start_Date,
    MAX(event_date) AS Max_Date,
    ROUND((MAX(CAST(event_timestamp AS INT64)) - MIN(CAST(event_timestamp AS INT64))) / (86400 * 1000000),1) AS Installed_Days,
    user_properties.value.string_value AS Username,
    MIN(user_first_touch_timestamp) = MIN(event_timestamp) AS first_touch_equal_first_event,
    MIN(user_first_touch_timestamp) AS first_touch_timestamp,
    MIN(event_timestamp) AS min_event_timestamp,
    MAX(event_timestamp) AS max_timestamp
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
WHERE
    user_first_touch_timestamp > 20181110
GROUP BY
    user_properties.value.string_value
ORDER BY
    (MAX(CAST(event_timestamp AS INT64)) - MIN(CAST(event_timestamp AS INT64))) DESC
"""
df = client.query(query).to_dataframe()
df.head(10)

,Min_Date,Max_Date,Active_Days,Username,first_touch_equal_first_event,first_touch_timestamp,min_event_timestamp,max_timestamp
0,20180711,20190123,196.2,None,False,1524757617404000,1531305681399000,1548257649805857
1,20180806,20181217,133.1,test,False,1524798650061000,1533604351828000,1545100704746055
2,20180917,20190122,127.0,test23,False,1536522983060000,1537238931729000,1548213752564001
3,20180918,20190123,127.0,emily,False,1537276848519000,1537276864561000,1548251351687729
4,20180919,20190123,126.1,liamkl,False,1537361098120000,1537361121551000,1548255184682384
5,20181108,20190122,75.4,chshanahan,False,1541689883839000,1541689955151000,1548207774634011
6,20181110,20190122,73.2,vinnov10,False,1541890948880000,1541891256054000,1548214997130003
7,20180906,20181117,71.9,test5,False,1536287952446000,1536290491856000,1542506112545066
8,20181112,20190123,71.6,Jake,False,1542041638325000,1542041672191000,1548225102724130
9,20181114,20190117,64.3,+vinny,False,1542208044439000,1542208107623000,1547767715989016


### 

In [28]:
query = """
SELECT
    MIN(event_date) AS Date,
    MIN(event_timestamp) AS min_event_timestamp,
    MIN(user_first_touch_timestamp) AS Install,
    device.advertising_id AS Device_ID
FROM
    `analytics_153084895.events_*`,
    UNNEST(user_properties) AS user_properties
GROUP BY
    device.advertising_id
"""
df = client.query(query).to_dataframe()
df.head(10)

,Date,min_event_timestamp,Install,Device_ID
0,20181123,1542983237762000,1542983237762000,37741aec-3cc6-466b-be4b-e06848021936
1,20181119,1542655221274000,1542655221274000,b0003bd6-2888-4f5e-abb4-05e34eafaccb
2,20180714,1531575491112000,1524757617404000,719bac60-d479-4a35-8c92-3773c759349a
3,20181223,1545615800164000,1545615800164000,e3e0e9fe-4c95-4aac-b9ca-51a7f3e55483
4,20181109,1541768409654000,1541768409654000,39378377-f0da-42c1-86df-cee45bfc58aa
5,20181124,1543052808229000,1543052808229000,8e0daa9e-2df5-4672-aad7-92c0eab7cd7b
6,20180723,1532392841763000,1531184677671000,c51ae1de-cae7-43c0-a0ed-e68909cefd0e
7,20181111,1541946427915000,1541946427915000,6149d95e-8be2-447b-b7a0-b40b8ae17609
8,20180919,1537361098120000,1537361098120000,c0431035-b543-400e-99e0-b6b9d78eed78
9,20181112,1542041638325000,1542041638325000,c3bb7aae-16ca-4720-bcd5-9d08b65247cf


In [ ]:
query = """
Template
"""
df = client.query(query).to_dataframe()
df.head(10)

In [ ]:
query = """
Template
"""
df = client.query(query).to_dataframe()
df.head(10)

In [ ]:
query = """
Template
"""
df = client.query(query).to_dataframe()
df.head(10)

In [ ]:
query = """
Template
"""
df = client.query(query).to_dataframe()
df.head(10)

In [ ]:
query = """
Template
"""
df = client.query(query).to_dataframe()
df.head(10)

In [ ]:
query = """
Template
"""
df = client.query(query).to_dataframe()
df.head(10)

In [ ]:
query = """
Template
"""
df = client.query(query).to_dataframe()
df.head(10)

In [ ]:
query = """
Template
"""
df = client.query(query).to_dataframe()
df.head(10)

In [ ]:
query = """
Template
"""
df = client.query(query).to_dataframe()
df.head(10)